In [1]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
import tqdm
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

In [2]:
df = pd.read_csv("C:/Users/Rebeca OdC/Documents/A4/Social media analysis/final_dataframe_4790_articles.csv")
#corpus_text = df['lead_paragraph'].values

In [3]:
df['full_text']=df['headline']+df['lead_paragraph']
corpus_text = df['full_text'].values
df=df.dropna(subset=['full_text'])
df['lenght']=df['full_text'].apply(len)

In [4]:
#CONVERTING INTO LIST OF TOKEMS AND LEMATIZATION
nlp = spacy.load("en_core_web_sm")
corpus_text=df.full_text.values
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)
docs = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]

#To avoid 'or' and 'of', I remove words that have 2 characters:
docs = [[token for token in doc if len(token) > 2] for doc in docs]

#CREATING DICTIONNARY
id2word = Dictionary(docs)
id2word.filter_extremes(no_below=20, no_above=0.1)
corpus = [id2word.doc2bow(text) for text in docs]
#testing dictionnary
print(corpus[100])

100%|██████████| 4778/4778 [00:40<00:00, 118.95it/s]


[(18, 1), (154, 1), (179, 1), (208, 1), (261, 2), (265, 1), (271, 1), (316, 1), (345, 1), (373, 1), (390, 1), (447, 1), (457, 1), (519, 1), (785, 1), (827, 1), (867, 1), (868, 1)]


In [5]:
#Finally, the model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=15,
                   passes=20,
                   chunksize=2000)
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

#docs_2_topics will store the probability distribution of topics over documents.
docs_2_topics = lda_model[corpus]
docs_2_topics[24]

[(4, 0.06422394),
 (5, 0.22576605),
 (6, 0.18367796),
 (8, 0.08306155),
 (11, 0.37078127),
 (14, 0.04940905)]

In [6]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 south, military, japan, asia, sea, pacific, two, between, defense, philippines
1 :	 russia, missile, russian, defense, putin, after, military, war, about, afghanistan
2 :	 kong, hong, who, state, she, bill, law, democracy, university, one
3 :	 here, sign, your, get, briefing, email, morning, india, pakistan, friday
4 :	 leader, jinping, south, kim, korean, jong, when, meeting, than, first
5 :	 taiwan, donald, clinton, state, beijing, between, secretary, hillary, elect, visit
6 :	 european, europe, union, currency, leader, stock, market, they, again, between
7 :	 more, oil, they, than, global, their, will, other, america, economic
8 :	 news, biden, presidential, foreign, policy, about, election, medium, times, debate
9 :	 house, official, white, state, against, pompeo, government, secretary, ban, mike
10 :	 deal, tariff, economic, war, between, impose, move, beijing, billion, good
11 :	 coronavirus, minister, climate, prime, global, change, pandemic, mexico, health, foreign
12 :	 i

In [7]:
pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
visualisation

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.060337  0.009928       1        1   8.412314
1      0.020610  0.141270       2        1   6.216677
2     -0.046816 -0.007524       3        1   5.403800
3      0.452123 -0.097456       4        1   3.000452
4     -0.093830 -0.146174       5        1   6.629875
5     -0.059236  0.021345       6        1   6.894483
6     -0.081727 -0.067226       7        1   5.412425
7     -0.105054 -0.046179       8        1  10.281816
8      0.027848 -0.122357       9        1   4.931977
9      0.059281  0.228874      10        1   6.202211
10    -0.012876  0.072511      11        1   7.532163
11    -0.004676 -0.069832      12        1   4.699117
12    -0.021433  0.143377      13        1   8.397762
13    -0.003782  0.118769      14        1   5.648199
14    -0.070094 -0.179326      15        1  10.336728, topic_info=        Term        Freq       Total Category  logprob  loglift
114     iran  633.000000  633.000000  Default  30.0000  30.0000
274  nuclear  586.000000  586.000000  Default  29.0000  29.0000
99      here  338.000000  338.000000  Default  28.0000  28.0000
92      sign  333.000000  333.000000  Default  27.0000  27.0000
420     kong  372.000000  372.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
73      long   75.506702  246.638751  Topic15  -4.8942   1.0858
218      get   77.046876  312.278536  Topic15  -4.8740   0.8700
31     their   84.726432  506.388103  Topic15  -4.7790   0.4816
110      day   76.851360  315.878121  Topic15  -4.8765   0.8560
46      will   82.466577  607.489649  Topic15  -4.8060   0.2725

[814 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
955       2  0.846971  abandon
955       3  0.105871  abandon
1155     12  0.960806      abe
81        1  0.059371    about
81        2  0.122452    about
...     ...       ...      ...
80        5  0.571344    young
80       15  0.091415    young
383       4  0.997763     your
984       2  0.818998     zone
984      13  0.153562     zone

[2255 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [8]:
pyLDAvis.save_html(visualisation, 'topic_modelling_visualisation.html')

In [9]:
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [10]:
#if we want to find the best article(s) for a topic
topic_number=1
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

russia, missile, russian, defense, putin, after, military, war, about, afghanistan
U.S. Says Russia Tested Missile, Despite TreatyWASHINGTON — The United States informed its NATO allies this month that Russia had tested a new ground-launched cruise missile, raising concerns about Moscow’s compliance with a landmark arms control accord.  0.9555534
Airlines Urged by U.S. to Give Notice to ChinaWASHINGTON — Even as China scrambled fighter jets to enforce its newly declared air defense zone, the Obama administration said on Friday that it was advising American commercial airlines to comply with China’s demands to be notified in advance of flights through the area.  0.94813937
The U.S. Needs More Nukes“In the capitals of the North Atlantic Treaty Organization, there is a clear lack of confidence in the United States’ reliability as a military ally.” 0.92221683
Decoding Trump’s Plan to Rein In ChinaBEIJING — It isn’t just about missiles and militaries anymore. 0.86665905
Kyrgyzstan Says It W